In [1]:
import numpy as np
import pandas as pd
from scipy import stats

#importing the models
import Kmeans
import ALS
import NN

Using TensorFlow backend.


In [3]:
training_path = "data/data_train.csv"
format_path = "data/sampleSubmission.csv"
NN.main(training_path, format_path)

/storage/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1059256 samples, validate on 117696 samples
Epoch 1/10
1059256/1059256 [==============================] - 51s 48us/step - loss: 1.0339 - val_loss: 1.0067
Epoch 2/10
 217600/1059256 [=====>........................] - ETA: 40s - loss: 1.0025

KeyboardInterrupt: 

In [ ]:
def median_vote(predictions):
    #computing the median
    median=np.median(predictions,axis=0)
    #making sure the result is an int (not the case if the amount of predictions is even)
    bounded_median=np.floor(median)
    return bounded_median

In [ ]:
def mode_vote(predictions):
    #computing the mode
    return stats.mode(predictions,axis=0)[0]

In [ ]:
def vote(voting_f):
    #useful constants
    submission_path='submission.csv'
    training_path = "data/data_train.csv"
    format_path = "data/sampleSubmission.csv"
    #Initializing 'predictions' and 'labels' using a first prediction
    predictions=ALS.main(training_path, format_path).to_numpy()
    labels=predictions[:,0]
    predictions=predictions[:,1]
    #computing multiple predictions of the kmeans algorithm
    for k in [3,6,7]:
        predictions=np.vstack((predictions,Kmeans.main(training_path, format_path, k).to_numpy()[:,1]))
    
    predictions=np.vstack((predictions, NN.main(training_path, format_path).to_numpy()[:,1]))
    
    #finding the best prediction though the voting function
    pred=pd.DataFrame(np.vstack((labels,voting_f(predictions))).T)
    pred=pred.rename(columns={0: 'Id', 1:'Prediction'})
    #exporting the final prediction using the submission path
    pred.to_csv(submission_path)

In [ ]:
vote(median_vote)